In [1]:
""" Goal: multisig acct with two signers, where either one 
can sign all transactions on behalf of the account.
Start by resetting the sandbox: ./sandbox reset

"""

""" In this cell, we set up the two accounts who will be the two signers
and fund them."""
from random import randint
from settings import Settings
import logging
import dev_setup
import base64
import util
from algosdk.v2client.algod import AlgodClient
from algosdk import encoding
from algosdk.future import transaction
from algosdk.future.transaction import Multisig, MultisigTransaction, wait_for_confirmation
settings = Settings()
logging.basicConfig(level='INFO')

oceanSk = 'sp4SDWmH8Rin0IhPJQq1UMsSR5C0j1IGqzLdcwCMySBVzT8lEUwjwwpS9z6l6dKSg52WWEjRdJDAL+eVt4kvBg=='
oceanAccount = util.AlgoAccount(privateKey=oceanSk)
txn_resp1 = dev_setup.devFundAccount(settings, acctToFund=oceanAccount, microAlgoAmount=1_000_000)

waveSk = 'FCLmrvflibLD6Deu3NNiUQCC9LOWpXLsbMR/cP2oJzH8IT4Zu8vBAhRNsXoWF+2i6q2KyBZrPhmbDCKJD7rBBg=='
waveAccount = util.AlgoAccount(privateKey=waveSk)
txn_resp2 = dev_setup.devFundAccount(settings, acctToFund=waveAccount, microAlgoAmount=1_000_000)
client: AlgodClient = util.getAlgodClient(settings)


INFO:util: ..KSISJM sending payment of 1.0 algo/s to ..DDX4BI 
 txID: ..5ON3KA
INFO:util:Got response for ..5ON3KA
INFO:util: ..KSISJM sending payment of 1.0 algo/s to ..WJ4MII 
 txID: ..I2OIQA
INFO:util:Got response for ..I2OIQA


In [2]:
# We have two accounts - ocean and wave. We make a multi-sig account where either can sign

msig = Multisig(version=1, threshold=1, addresses=[oceanAccount.addr, waveAccount.addr])
assert msig.threshold == 1
subsigOcean: transaction.MultisigSubsig = msig.subsigs[0]
assert isinstance(subsigOcean, transaction.MultisigSubsig)
subsigWave: transaction.MultisigSubsig = msig.subsigs[1]
assert subsigOcean.public_key == transaction.MultisigSubsig(encoding.decode_address(oceanAccount.addr)).public_key
assert subsigWave.public_key == transaction.MultisigSubsig(encoding.decode_address(waveAccount.addr)).public_key

assert msig.address()[-6:] == 'KD2SVI'
# Fund the multisig account from a genesis account
gen = dev_setup.devGetGenesisAccounts(settings)[0]
txn = transaction.PaymentTxn(
    sender=gen.addr,
    receiver=msig.address(),
    amt=500_000,
    sp=client.suggested_params()
)
signedTxn = txn.sign(gen.sk)
txid = client.send_transaction(signedTxn)
confirmed_txn = wait_for_confirmation(client, txid, 6)  
assert client.account_info(msig.address())['amount'] >= 500_000


In [3]:
success1, txn1, mtx1 = dev_setup.multiMakesNft(msig=msig, subAcct=oceanAccount)

Sent. Waiting for confirmation
10
success sending
{'subsig': [{'pk': 'Vc0/JRFMI8MKUvc+penSkoOdllhI0XSQwC/nlbeJLwY=', 's': 'ozlZVKGXDx7+eNr4g4EfNeMxrKvzaP4QFvJNIP2jKgLotIt5G4T/udVUYhU6Coac6+uY1wMxcu4S5kwgEDhMBw=='}, {'pk': '/CE+GbvLwQIUTbF6FhftouqtisgWaz4ZmwwiiQ+6wQY='}], 'thr': 1, 'v': 1}


In [5]:
success2, txn2, mtx2 = dev_setup.multiMakesNft(msig=msig, subAcct=waveAccount)

AlgoHTTPError: At least one signature didn't pass verification
{'subsig': [{'pk': 'Vc0/JRFMI8MKUvc+penSkoOdllhI0XSQwC/nlbeJLwY=', 's': 'ozlZVKGXDx7+eNr4g4EfNeMxrKvzaP4QFvJNIP2jKgLotIt5G4T/udVUYhU6Coac6+uY1wMxcu4S5kwgEDhMBw=='}, {'pk': '/CE+GbvLwQIUTbF6FhftouqtisgWaz4ZmwwiiQ+6wQY=', 's': '7ny/9djVNJmZPOWNm4LrFLyFdJm1wPob+XlSIq1LCTuK/CDQiUN51Fx+Gt3UOM2ke69heW1WJ5sUEXqY/MirDA=='}], 'thr': 1, 'v': 1}


In [ ]:
assert success2 == False

In [16]:
txn2.asset_name, txn2.note, txn2.unit_name

('SeaWave384',
 b'\xd6\xfd\xf7\xb0\xe2w\xa8\x90^\xf8\xe4\xcd\x9c\x0e\xbc\xbf\xa4\xf1Pn',
 'W384')

In [8]:
assert txn1.note != txn2.note
assert txn1.asset_name != txn2.asset_name
assert txn1.unit_name != txn2.unit_name

In [9]:
success3, txn3, stx3 = dev_setup.singleMakesNft(acct=oceanAccount)

Sent. Waiting for confirmation
11
success sending


In [12]:
success4, txn4, stx4 = dev_setup.singleMakesNft(acct=waveAccount)

Sent. Waiting for confirmation
12
success sending


In [14]:
assert stx3.signature != stx4.signature

In [17]:
txn4.asset_name, txn4.note, txn4.unit_name

('SeaWave87', b'GO\x19\xac6\xb1LpSfr\xb3\x94\xe3kP\xd2\xee c', 'W87')